In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import joblib
from sklearn.ensemble import RandomForestClassifier


In [3]:
data_path = "data/"
data_file = "competencia_01.csv"
df= pd.read_csv(data_path + data_file)

C:\Users\chris\AppData\Local\Temp\ipykernel_14112\4193464990.py:3: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_path + data_file)


In [ ]:
cat_cols =[]
num_cols=[]
col_drops=["numero_de_cliente","foto_mes","active_quarter","clase_ternaria","cliente_edad","cliente_antiguedad","active_quarter"]
for c in df.columns:
    if (df[c].nunique() <= 5):
        cat_cols.append(c)
    else:
        num_cols.append(c)
lista_t=[c for c in list(map(lambda x : x if x[0]=='t' and x not in col_drops else np.nan ,df.columns )) if pd.notna(c)]
lista_c=[c for c in list(map(lambda x : x if x[0]=='c' and x not in col_drops else np.nan ,df.columns )) if pd.notna(c)]
lista_m=[c for c in list(map(lambda x : x if x[0]=='m' and x not in col_drops else np.nan ,df.columns )) if pd.notna(c)]
lista_r=[c for c in df.columns if c not in (lista_t + lista_c + lista_m +col_drops )]


# # Columnas lags y delta
cols_lag_delta_max_min_regl=lista_m + lista_c+ lista_r+lista_r

# # Columnas para regresion lineal y max-min
# lista_regl_max_min = lista_m + lista_c+ lista_r+lista_r

# # Columnas para los ratios
cols_ratios=[]
for c in lista_c:
    i=0
    while i < len(lista_m) and c[1:] != lista_m[i][1:]:
        i+=1
    if i < len(lista_m):
        cols_ratios.append([lista_m[i],c ])



In [38]:
"foto_mes" in cols_lag_delta_max_min_regl

False

In [37]:
len(cols_ratios)

23

In [33]:
cols_ratios

[['mcuenta_corriente', 'ccuenta_corriente'],
 ['mcaja_ahorro', 'ccaja_ahorro'],
 ['mprestamos_personales', 'cprestamos_personales'],
 ['mprestamos_prendarios', 'cprestamos_prendarios'],
 ['mprestamos_hipotecarios', 'cprestamos_hipotecarios'],
 ['minversion2', 'cinversion2'],
 ['mcuenta_debitos_automaticos', 'ccuenta_debitos_automaticos'],
 ['mpagodeservicios', 'cpagodeservicios'],
 ['mpagomiscuentas', 'cpagomiscuentas'],
 ['mcajeros_propios_descuentos', 'ccajeros_propios_descuentos'],
 ['mtarjeta_visa_descuentos', 'ctarjeta_visa_descuentos'],
 ['mtarjeta_master_descuentos', 'ctarjeta_master_descuentos'],
 ['mcomisiones_mantenimiento', 'ccomisiones_mantenimiento'],
 ['mcomisiones_otras', 'ccomisiones_otras'],
 ['mforex_buy', 'cforex_buy'],
 ['mforex_sell', 'cforex_sell'],
 ['mtransferencias_recibidas', 'ctransferencias_recibidas'],
 ['mtransferencias_emitidas', 'ctransferencias_emitidas'],
 ['mextraccion_autoservicio', 'cextraccion_autoservicio'],
 ['mcheques_depositados', 'ccheques_dep

In [30]:
len(cols_ratios)

23

In [19]:
df.shape

(978439, 155)

In [24]:
lista_c+lista_m + lista_t +lista_r

['cliente_vip',
 'cproductos',
 'ccuenta_corriente',
 'ccaja_ahorro',
 'cdescubierto_preacordado',
 'ctarjeta_debito',
 'ctarjeta_debito_transacciones',
 'ctarjeta_visa',
 'ctarjeta_visa_transacciones',
 'ctarjeta_master',
 'ctarjeta_master_transacciones',
 'cprestamos_personales',
 'cprestamos_prendarios',
 'cprestamos_hipotecarios',
 'cplazo_fijo',
 'cinversion1',
 'cinversion2',
 'cseguro_vida',
 'cseguro_auto',
 'cseguro_vivienda',
 'cseguro_accidentes_personales',
 'ccaja_seguridad',
 'cpayroll_trx',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'ctarjeta_visa_debitos_automaticos',
 'ctarjeta_master_debitos_automaticos',
 'cpagodeservicios',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'ctarjeta_master_descuentos',
 'ccomisiones_mantenimiento',
 'ccomisiones_otras',
 'cforex',
 'cforex_buy',
 'cforex_sell',
 'ctransferencias_recibidas',
 'ctransferencias_emitidas',
 'cextraccion_autoservicio',
 'ccheques_depositados',
 'ccheques_emitidos',

In [27]:
sorted(cat_cols)

['Master_delinquency',
 'Master_status',
 'Visa_delinquency',
 'Visa_status',
 'active_quarter',
 'ccajas_transacciones',
 'ccuenta_corriente',
 'cdescubierto_preacordado',
 'clase_ternaria',
 'cliente_vip',
 'cmobile_app_trx',
 'cseguro_accidentes_personales',
 'cseguro_vida',
 'ctarjeta_master',
 'ctarjeta_visa',
 'internet',
 'tcallcenter',
 'tcuentas',
 'thomebanking',
 'tmobile_app']

In [13]:
lista_t

['tcuentas', 'tcallcenter', 'thomebanking', 'tmobile_app']

In [14]:
lista_c

['cliente_vip',
 'cproductos',
 'ccuenta_corriente',
 'ccaja_ahorro',
 'cdescubierto_preacordado',
 'ctarjeta_debito',
 'ctarjeta_debito_transacciones',
 'ctarjeta_visa',
 'ctarjeta_visa_transacciones',
 'ctarjeta_master',
 'ctarjeta_master_transacciones',
 'cprestamos_personales',
 'cprestamos_prendarios',
 'cprestamos_hipotecarios',
 'cplazo_fijo',
 'cinversion1',
 'cinversion2',
 'cseguro_vida',
 'cseguro_auto',
 'cseguro_vivienda',
 'cseguro_accidentes_personales',
 'ccaja_seguridad',
 'cpayroll_trx',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'ctarjeta_visa_debitos_automaticos',
 'ctarjeta_master_debitos_automaticos',
 'cpagodeservicios',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'ctarjeta_master_descuentos',
 'ccomisiones_mantenimiento',
 'ccomisiones_otras',
 'cforex',
 'cforex_buy',
 'cforex_sell',
 'ctransferencias_recibidas',
 'ctransferencias_emitidas',
 'cextraccion_autoservicio',
 'ccheques_depositados',
 'ccheques_emitidos',

In [15]:
lista_m

['mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheque

In [18]:
lista_r

['internet',
 'Master_delinquency',
 'Master_status',
 'Master_mfinanciacion_limite',
 'Master_Fvencimiento',
 'Master_Finiciomora',
 'Master_msaldototal',
 'Master_msaldopesos',
 'Master_msaldodolares',
 'Master_mconsumospesos',
 'Master_mconsumosdolares',
 'Master_mlimitecompra',
 'Master_madelantopesos',
 'Master_madelantodolares',
 'Master_fultimo_cierre',
 'Master_mpagado',
 'Master_mpagospesos',
 'Master_mpagosdolares',
 'Master_fechaalta',
 'Master_mconsumototal',
 'Master_cconsumos',
 'Master_cadelantosefectivo',
 'Master_mpagominimo',
 'Visa_delinquency',
 'Visa_status',
 'Visa_mfinanciacion_limite',
 'Visa_Fvencimiento',
 'Visa_Finiciomora',
 'Visa_msaldototal',
 'Visa_msaldopesos',
 'Visa_msaldodolares',
 'Visa_mconsumospesos',
 'Visa_mconsumosdolares',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_madelantodolares',
 'Visa_fultimo_cierre',
 'Visa_mpagado',
 'Visa_mpagospesos',
 'Visa_mpagosdolares',
 'Visa_fechaalta',
 'Visa_mconsumototal',
 'Visa_cconsumos',
 'Visa

In [11]:
cat_cols

['active_quarter',
 'cliente_vip',
 'internet',
 'tcuentas',
 'ccuenta_corriente',
 'cdescubierto_preacordado',
 'ctarjeta_visa',
 'ctarjeta_master',
 'cseguro_vida',
 'cseguro_accidentes_personales',
 'tcallcenter',
 'thomebanking',
 'ccajas_transacciones',
 'tmobile_app',
 'cmobile_app_trx',
 'Master_delinquency',
 'Master_status',
 'Visa_delinquency',
 'Visa_status',
 'clase_ternaria']

In [7]:
lista_c

['cliente_edad',
 'cliente_antiguedad',
 'cproductos',
 'ccaja_ahorro',
 'ctarjeta_debito',
 'ctarjeta_debito_transacciones',
 'ctarjeta_visa_transacciones',
 'ctarjeta_master_transacciones',
 'cprestamos_personales',
 'cprestamos_prendarios',
 'cprestamos_hipotecarios',
 'cplazo_fijo',
 'cinversion1',
 'cinversion2',
 'cseguro_auto',
 'cseguro_vivienda',
 'ccaja_seguridad',
 'cpayroll_trx',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'ctarjeta_visa_debitos_automaticos',
 'ctarjeta_master_debitos_automaticos',
 'cpagodeservicios',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'ctarjeta_master_descuentos',
 'ccomisiones_mantenimiento',
 'ccomisiones_otras',
 'cforex',
 'cforex_buy',
 'cforex_sell',
 'ctransferencias_recibidas',
 'ctransferencias_emitidas',
 'cextraccion_autoservicio',
 'ccheques_depositados',
 'ccheques_emitidos',
 'ccheques_depositados_rechazados',
 'ccheques_emitidos_rechazados',
 'ccallcenter_transacciones',
 'chomebanking_t

In [6]:
lista_r

['numero_de_cliente',
 'foto_mes',
 'Master_mfinanciacion_limite',
 'Master_Fvencimiento',
 'Master_Finiciomora',
 'Master_msaldototal',
 'Master_msaldopesos',
 'Master_msaldodolares',
 'Master_mconsumospesos',
 'Master_mconsumosdolares',
 'Master_mlimitecompra',
 'Master_madelantopesos',
 'Master_madelantodolares',
 'Master_fultimo_cierre',
 'Master_mpagado',
 'Master_mpagospesos',
 'Master_mpagosdolares',
 'Master_fechaalta',
 'Master_mconsumototal',
 'Master_cconsumos',
 'Master_cadelantosefectivo',
 'Master_mpagominimo',
 'Visa_mfinanciacion_limite',
 'Visa_Fvencimiento',
 'Visa_Finiciomora',
 'Visa_msaldototal',
 'Visa_msaldopesos',
 'Visa_msaldodolares',
 'Visa_mconsumospesos',
 'Visa_mconsumosdolares',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_madelantodolares',
 'Visa_fultimo_cierre',
 'Visa_mpagado',
 'Visa_mpagospesos',
 'Visa_mpagosdolares',
 'Visa_fechaalta',
 'Visa_mconsumototal',
 'Visa_cconsumos',
 'Visa_cadelantosefectivo',
 'Visa_mpagominimo']

In [5]:
cols_lag_delta

['mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheque

In [ ]:
with open("outputs/rf_model__completo_2025-09-20_00-11-18_2025-09-20 00_38_32.sav",)